In [83]:
import pandas as pd
import numpy as np
import copy as cp
import darc_core
import pickle
import sys
import hashlib as hs
import base64 as b64
import os
import random as rd
import datetime as dt
from darc_core.metrics import Metrics
from darc_core.utils import M_COL, T_COL, T_COL_IT, NB_GUESS, SIZE_POOL
from darc_core.preprocessing import round1_preprocessing
from darc_core.utils import check_format_trans_file

In [160]:
%%time
df = pd.read_csv("./ground_truth.csv", parse_dates=["date"])
df["hours"]= pd.to_datetime(df["hours"], format = '%H:%M%S').dt.hour
df["month"]=df["date"].dt.month
df["year"]=df["date"].dt.year
df["id_user"]=df["id_user"].astype(str)
df["price"]=df["price"].astype("float")
df["qty"]=df["qty"].astype("float")
df["index"]=df.index
df["year_cat"]=df["year"].astype("category")
df["month_cat"]=df["month"].astype("category")
df_dict = df.T.to_dict('list')
print(df.head(3))

  id_user       date  hours id_item  price  qty  month  year  index year_cat  \
0   17850 2010-12-01      8  85123A   2.55  6.0     12  2010      0     2010   
1   17850 2010-12-01      8   71053   3.39  6.0     12  2010      1     2010   
2   17850 2010-12-01      8  84406B   2.75  8.0     12  2010      2     2010   

  month_cat  
0        12  
1        12  
2        12  
CPU times: user 45.6 s, sys: 625 ms, total: 46.3 s
Wall time: 47.6 s


In [ ]:
dfn[(dfn["id_item"]==dfn["id_item"][0]) & (dfn["month"]==12) & (dfn["year"]==2010)].head(30)

In [182]:
dfn.loc[finished_partitions[0]]

,id_user,date,hours,id_item,price,qty,month,year,index,9,10
4556,13069,2010-12-01,12.1667,10002,0.85,15.4167,12,2010,4556,2010,12
31,12583,2010-12-03,12.1667,10002,0.85,15.4167,12,2010,31,2010,12
12123,12731,2010-12-09,12.1667,10002,0.85,15.4167,12,2010,12123,2010,12
15645,16805,2010-12-06,12.1667,10002,0.85,15.4167,12,2010,15645,2010,12
106,16098,2010-12-14,12.1667,10002,0.85,15.4167,12,2010,106,2010,12
18507,14713,2010-12-16,12.1667,10002,0.85,15.4167,12,2010,18507,2010,12
6454,17677,2010-12-13,12.1667,10002,0.85,15.4167,12,2010,6454,2010,12
3721,17967,2010-12-01,12.1667,10002,0.85,15.4167,12,2010,3721,2010,12
14196,16701,2010-12-09,12.1667,10002,0.85,15.4167,12,2010,14196,2010,12
11366,16795,2010-12-05,12.1667,10002,0.85,15.4167,12,2010,11366,2010,12


In [161]:
%%time
def mergeIdentical(df,df_dict):
    """ Merge "same" transaction into one line. Assign "DEL" to duplicates
        Transactions are "identical" if their keys are equal
        
        :map: key:[id_user,date,id_item] value:[qty_cumulative,count]
    """
    map={}
    for d in range(0,len(df)):
        if(map.get((df["id_user"][d],df["date"][d],df["id_item"][d]))):
            map[(df["id_user"][d],df["date"][d],df["id_item"][d])][0] += df_dict[d][5]
            map[(df["id_user"][d],df["date"][d],df["id_item"][d])][1] += 1
            df_dict[d][0]="DEL"
            df_dict[d][1]=df_dict[d][1].strftime("%Y-%m-01")
            df_dict[d][2]=""
            df_dict[d][3]=""
            df_dict[d][4]=""
            df_dict[d][5]=""
        else:
            map[(df["id_user"][d],df["date"][d],df["id_item"][d])] = [df_dict[d][5],1]
    
    dfn = pd.DataFrame.from_dict(df_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year", 8:"index",9:"year_cat",10:"month_cat"})    
  
    return dfn ,map

dfn, map= mergeIdentical(df,df_dict)
save_dfn_merge=dfn.copy()

CPU times: user 39.7 s, sys: 451 ms, total: 40.2 s
Wall time: 41.5 s


In [11]:
def split(df, partition, column): 
    """
        Split df[partition] over column on median 
        :return: List[[IndexRange]]*2 {left ,right}
    """
    dfp = df["id_user"][partition]
    sanitizedPartition = dfp.index[dfp != "DEL"]
    dfp = df[column][sanitizedPartition]
    if (column in categorical):
        values = dfp.unique()
        dfp.sort_values(inplace=True)
        lv=set(values[:len(values)//2])
        rv=set(values[len(values)//2:])
        return dfp.index[dfp.isin(lv)],dfp.index[dfp.isin(rv)]
    else:
        median=dfp.median()
        dfp.sort_values(inplace=True)
        dfl=dfp.index[dfp<median]
        dfr=dfp.index[dfp>=median]
        return(dfl,dfr)

In [154]:
def is_k_anonymous (df, partition, k=1):
    if ((len(partition)<k)):
        return False
    return True

In [162]:
%%time
dfn["id_item"]=dfn["id_item"].astype("category")
categorical = set ({'id_item', 'year_cat', 'month_cat'})
def partition_dataset(df, is_valid):
    """
        Greedy search
        Split dataframe over {year,month,id_item} 
        :return: List[[IndexRange]] 
    """
    finished_partitions=[]
    partitions=[df.index]
    while partitions:    
        partition = partitions.pop(0)
        columns = ["year_cat","month_cat","id_item"]
        for column in columns:
            lp, rp = split(df,partition,column)
            if ( (not is_valid(df,lp)) or (not is_valid(df,rp)) ):
                continue
            partitions.extend((lp,rp))
            break
        else:
            finished_partitions.append(partition)
    return finished_partitions
finished_partitions = partition_dataset(dfn, is_k_anonymous)
save_finished_partitions=finished_partitions.copy()

CPU times: user 15min 45s, sys: 8.33 s, total: 15min 53s
Wall time: 20min 20s


In [163]:
with open('finished_partitions', 'wb') as fp:
    pickle.dump(finished_partitions, fp)

In [164]:
with open ('finished_partitions', 'rb') as fp:
    finished_partitions = pickle.load(fp)

In [35]:
array = [0]*9
for f in finished_partitions:
    if(f.nunique()<9):
        array[f.nunique()]+=1

In [36]:
array

[0, 5238, 3199, 2199, 1689, 1412, 1173, 1056, 923]

In [96]:
%%time
def make_l_diverse(df_dict,partitions,l=9):
    filtered=[]
    for p in finished_partitions:
        array = []
        for i in p:
            if(df_dict[i][0] != "DEL" and df_dict[i][0] not in array):
                array.append(df_dict[i][0])
        if len(array)<l:
            for i in p:
                
                if(df_dict[i][6]==12 and df_dict[i][7]==2010):
                    free_space[0].append(i)    
                else:
                    free_space[df_dict[i][6]].append(i)
                    
                df_dict[i][0]="DEL"
                df_dict[i][1]=df_dict[i][1].strftime("%Y-%m-01")
                df_dict[i][2]=""
                df_dict[i][3]=""
                df_dict[i][4]=""
                df_dict[i][5]=""
        else:
            filtered.append(p)
            
    dfn = pd.DataFrame.from_dict(df_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year", 8:"index"})  
    
    return dfn,filtered

dfn, finished_partitions = make_l_diverse(df_dict,finished_partitions)
save_diversity=finished_partitions.copy()

CPU times: user 1.17 s, sys: 34 ms, total: 1.21 s
Wall time: 1.29 s


In [59]:
def generatePrices():
    itemPrices = {}
    items=dfn["id_item"].unique().tolist()
    
    for i in items:
        for d in range(0,len(df_dict)):
            if(df_dict[d][3]==i):
                itemPrices[i]= df_dict[d][4]
                break
    return itemPrices

itemPrice = generatePrices()

In [192]:
dfn=save_copy.copy()
df_dict=save_dict.copy()
def generate_month_util(dfn):
    months = list(range(0,13))
    month_items=[[]]*13
    month_users=[[]]*13
    for m in months:
        if(m==12):
            month_items[0]=dfn[(dfn["month"]==m) & (dfn["year"]==2010)]["id_item"].unique()  
            month_items[12]=dfn[(dfn["month"]==m) & (dfn["year"]==2011)]["id_item"].unique()
            month_users[0]=dfn[(dfn["month"]==m) & (dfn["year"]==2010)]["id_user"].unique()
            month_users[0]= month_users[0][month_users[0]!="DEL"]
            month_users[12]=dfn[(dfn["month"]==m) & (dfn["year"]==2011)]["id_user"].unique()
            month_users[12]= month_users[m][month_users[12]!="DEL"]
        else:
            month_items[m]=dfn[(dfn["month"]==m)]["id_item"].unique()
            month_users[m]=dfn[(dfn["month"]==m)]["id_user"].unique()
            month_users[m]= month_users[m][month_users[m]!="DEL"]

    return month_items, month_users

month_items, month_users = generate_month_util(dfn)

In [208]:
def generate_item_user_month(dfn):
    items_user_month = {}
    months = list(range(0,13))
    users = dfn["id_user"].unique()
    for i in range(0,len(df_dict)):
        if(df_dict[i][6]==12 and df_dict[i][7]==2010):
            m=0
        else:
            m=df_dict[i][6]
        u=df_dict[i][0]
        if(u != "DEL"):
            if(items_user_month.get((u,m))):
                items_user_month[(u,m)].append(df_dict[i][3])
            else:
                items_user_month[(u,m)]=[df_dict[i][3]]
    return items_user_month       
            
items_user_month = generate_item_user_month(dfn)        

In [46]:
%%time
dfn=save_copy.copy()
df_dict=save_dict.copy()

def generateDiff(user,month):
    return list(set(month_items[month])-set(items_user_month[(user,month)]))
    
def make_noise(df_dict,items_user_month):
    diff = {}
    for month,indexes in free_space.items():
        free_indexes=indexes.copy()
        while(free_indexes):
            for u in month_users[month]:
                if(diff.get((u,month)) and free_indexes): 
                    index = free_indexes.pop(0)
                    chosen_item = diff[(u,month)].pop(0)
                    df_dict[index][0]=u
                    df_dict[index][2]=rd.randrange(14)
                    df_dict[index][3]=chosen_item
                    df_dict[index][4]=itemPrice[chosen_item]
                    df_dict[index][5]=rd.randrange(10)
                elif(free_indexes):
                    diff[(u,month)]=generateDiff(u,month)
    dfn = pd.DataFrame.from_dict(df_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year", 8:"index"}) 
    return dfn

dfn = make_noise(df_dict,items_user_month)

NameError: name 'save_copy' is not defined

In [92]:
def shuffleDates(df,partition):
    date_array = []
    date_shuffled = []
    for i in partition:
        date_array.append(df_dict[i][1])
    while(len(date_array)):
        i=rd.randrange(len(date_array))
        date_shuffled.append(date_array[i])
        date_array.pop(i)
    k=0
    for i in partition:
        df_dict[i][1]=date_shuffled[k]
        k+=1
    
    return 1

In [172]:
%%time
def build_anonymized_dataset(df,partitions): 
    for p in partitions:
        mean_hour=0
        mean_qty=0
        for i in p:
            mean_hour+=df_dict[i][2]
            mean_qty+=df_dict[i][5]
            shuffleDates(df,p)
        for i in p:
            df_dict[i][2]=mean_hour/len(p)
            df_dict[i][5]=mean_qty/len(p)
    dfn = pd.DataFrame.from_dict(df_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year",8:"index"})  
    
    return dfn
dfn=build_anonymized_dataset(dfn,finished_partitions)
save_builded=dfn.copy()

CPU times: user 31.8 s, sys: 536 ms, total: 32.3 s
Wall time: 35.8 s


In [181]:
dfn=save_builded.copy()

In [100]:
gt = pd.read_csv("./ground_truth.csv", parse_dates=["date"])
gt["hours"]= pd.to_datetime(gt["hours"], format = '%H:%M%S').dt.hour
gt["month"]=gt["date"].dt.month
gt["year"]=gt["date"].dt.year
gt["id_user"]=gt["id_user"].astype(str)
gt["price"]=gt["price"].astype("float")
gt["qty"]=gt["qty"].astype("float")

In [ ]:
def parse_dates(date):
    if(isinstance(date,dt.date) and not pd.isnull(date)):
        return date.strftime('%Y/%m/%d')
dfn["date"]=dfn["date"].astype("datetime64[ns]")
dfn["date"]=dfn["date"].apply(lambda x: x.strftime('%Y/%m/%d'))
dfn.sort_values(["index"],inplace=True)
gt["date"]=gt["date"].astype("datetime64[ns]")

In [ ]:
for p in finished_partitions:
        m=df_dict[p[0]][1].month
        for d in p:
            if(df_dict[d][1].month != m):
                print("Expected month :", m, " Real Month :", df_dict[d][1].month ," Occured at index: ", df_dict[d][8])

In [165]:
for p in finished_partitions:
    if(df.loc[p,"month"].nunique()!=1):
        print(df.loc[p])

In [54]:
%%time
dfn.id_user=dfn.id_user.astype(str)
dfn_copy=dfn.copy()

def generateSalts():
    """ 
        Generate Salt{256b} Table : Mij = Salt for user i in month j 
    """
    ordered_months = list(range(1,14))
    ids = dfn_copy["id_user"].unique()
    ids = ids[ids != "DEL"]
    salt_table=pd.DataFrame(columns=ordered_months,index=ids)
    salt_table.set_index(ids,inplace=True)
    for i in ids:
        for j in ordered_months:
            # Generates unique b64 values
            salt_table[j][i]=os.urandom(256)
    return salt_table

salt_table = generateSalts()
salt_dict = salt_table.T.to_dict('list')

CPU times: user 14.4 s, sys: 2.2 s, total: 16.6 s
Wall time: 23.6 s


In [55]:
salt_table

,1,2,3,4,5,6,7,8,9,10,11,12,13
17850,b'=\x92`\xef\x0ew\xec\xf8\xf4N1m\xd3>\x1aLFC\x...,b'`\x0c$.\x1dTZ\xd5\x02\xe5\xc7\xa8\xe3\x1b\xc...,"b""\xae\xbd\x06o\x7fG\x06\x14\xf2\x98\xac\xefkw...",b'\xd9\xbe\xb3\xd3B\xa4\xcc/\x7f\x96]\x1e\xadf...,b'\xc3\xbb\xb9\x0f\x11\\\x8ep\\\xc0\xb8\xf4\xe...,"b""\xfb\xdc\xadio\xcf\xc7\x04\x8bN\xd2\x92\x9f\...",b'W\xe5iicc\xb7\xa9\xcf\xcb\xe1\xack\xa01G\xca...,"b'%Y\x93\x13""\x15\xb27\xd8\xf9\xa6\x00[\x06\\\...",b'`Bns\x8c\x18\xcf]\xc5\\\xdb\xbc\x07h+\xd7g\x...,"b""\xfa\xaa\xce\xd5\x00\xa8\xf0-\xab~\xb6\x90(\...","b""\xb5M'\x0b\xcd\xa9\x971\xe98\xa4R)-I6\xf9\x0...","b'\xec\xb4A\x8d\xd2,F\x8d\x00\xc8\xaaG\xfc\xf0...",b'\xa7\x83\x98(\xa1\xf3w\xb2\xe6\r\xb7\x1bAs\x...
13047,b's\xadbr\xb8\xc7#\xb5\xd5\xa2\xc5\x01i\xf9\xe...,b'\xbe\x1e8t\xb3\xb6\x1c\xefvQ\xd1\x1c`\x88\x9...,b'\x1c0\xaeB\xef\xcc\x8b\x16E`\x1f$~\xbd\xe0\x...,"b'u\x17\x026\x84,\xd5r\x94\x8f\x8d\xe8\x11z\x1...","b""\x04\x1b\xbcn\x81\xaa\xc9\xc5\x11\xe9\x03\xf...",b'+\x1e~\x152\xfb\x90\xa7#_\xaf?\xea\xd7\x1b\x...,b'\xb3\xc5\xce5W%p\xd6\xd0\x92\x18\xdbt\x1d\xf...,b'7{4\x80\x97\xaf\xb4{\x05A\x08\xd4k\x9a\x10(W...,b'wn\xea?4\x04-m5\x1cG\xea\xdb\xab\xa7%fC\xff\...,"b""\x8d\xbe\x9e\x87\xee\x8c6s\xfb\xae\xdc\xf2\x...",b'v(*3R\x97vF\xc3\x9c\xe5p\n\xb1\xa6W]\x7f\xd1...,b'\xb0\xd3\x89\xc1\x83^D\xb0x\xc9\xfe\x1b!%/\x...,b'\x0co@\xb1\x10@\xf6\xa8\x16\xaa\x03\x0c\t\xa...
12583,b'\x8c\x8f\xf7s\xf8k\xab\xda\x9b\xdaH\xa6\xfc@...,b'\xe7\xc0\xa7\x1fa\xb1\x96\xb9h\xc2/w\x01\xe5...,b'<\x8b\x91\x8b\x19U\xd0wcM\xfa\xfc\xde\xb1\x1...,b'\x93y\xa4t?\xd6\x16g\xc67\x02\xac^xW\x1ax\xd...,b'\xdf\x9e.j\x1c\xbb\xdeL\xd5\xa5`\x86>Z5~\x99...,b'\xe98\xfen\xf5\xdc\xa9\x90\x03\nKZD9JB;gR\xc...,b'V9\x92\xc0\xec#\xdf\xd2\x95\xa4\xaa\x97\xc1K...,b'v%\x1f\x88\xb7\x1f_\xda\x97d\x8e\xfc\x11\xd9...,b'\xb9\x12Q\xff\xfck\x02X\x14\xe4\xa46\xa34v0U...,b'\xf7S\xbeE\'\xe0G \xa8\xdaCO\xb5A\x07\x81\x1...,b'\xe2\xfd\xd3\xa3\x1a\x9f\x17\r\xa1zQ}\xb7Cjd...,b'\x86\xcb\xd2\x1c\xb9\xc3\xf8Yb\x84J\xd5EkN\x...,"b'\xdc\xb4\xb5\xc6\xdb\xfc\x1f%\xed\x14\xc9,g=..."
13748,b'\xf9\xa1\xe5\x9f\xa4gv\x16\x91`\xcc\x0cN\xa3...,b'\x01\xca~\x94b\xd3\x1dg\x96\xcd\x19\xc9\xa4\...,b'\x187\xc9\xea\xf8\x83F\xca\xc7\x11Z^\x90\xe2...,b'\x16\x05\x06\xbc\x8ckv\x0c\x9c\x13\xca\xea\x...,b'\xfd\x1c\xb4Yl\x1d\xe7N\xf2\xa8\xf0\x03n\xb5...,"b""]\x07\x17\x11\xac\xfd\xdf\xff\x8e\xd7^ \xfeA...",b'{\xac\x8dbJ\xf6Ii\xce\x16\x85n\x91\x00\xd1\x...,b'}(\xcd-(\xb9\xde\xd4\xb3{y\x82O\xdaz\xd4(M\x...,b'K\xf5\xf7\x037\x83\x16\x95\xe9T\x9a\xed\xf9\...,b'\xba\x83c\xf4\xd9\xb3\xdd#\x8ez\xc3\x04\x05\...,"b""\x84x\x8f\xc2#\xcbyr\xd1W\xa0`Q\x92\xfa\xc0\...","b""\xb5\x15\xc68\xb4\x93\x90/;Z \xfb\xcdw'\x88+...","b""6w+\x1c\xb1\xc6\xcam\xad\x80\xf4\x0c\x0e\xda..."
15100,b'\xaa\xca\x12:\xd4\x93\x19\x03m*\xf1\xab\xb79...,b'\x025\x83\xcb\xf4\x92u+\xbe\xf1&\x92\xd6\xff...,b')N\xd6\x9do\\\xfcP}o\x0eI\x02(Uy\xfcK\x0e\xf...,b'\xd0\xd2\xb8x\x94/7Zg*5 \xfb\xbez\x92a\xe9\x...,"b'\x9f\x94Q\x0e\xae;\xbe\xe0@7,\x02\x94cn\x0cU...",b'B\xad\x86lc\x0f\xb12\xd9g\xc7\ra\xafVr\x0b<-...,"b""#\x0bJ\xcfh\\Gd\x81&fh'VQ\x0b4\x12#}\xe7\x81...","b""(\xd6Sr\x8d\xd7\xaa\x84g\x90\xe9<_|]\x83\xbd...",b'\x93{\xbb\xb4\x86OP\xac\x02)~A\xda\xc0\x08\x...,b'~\x85\xd2\\gs\x15~f\xb1\xa5\xb8@\x9e\x98\x93...,b'\x9c\x8el\xa0\xec\xa1O[`\xe5\xd2\xad\xd2u4\x...,b'\x1b\xe8(\xfbu\x129\xb8\xc2J\x12\x0e\n@\xfb\...,"b""u\xca\x16\x10\xc5\x1d\xf3M\xf4\x92\x8d\x9e\x..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15471,b'\x0bz\xc4\xb2\x07K\xdb\x89.\x1d\xbc\xc9\x8b\...,b'\x14-\xf7s\xa5~\xc6\x08\x19\xe5\n\xec\xa2&;\...,"b""\xa6\xdb3\xdc\xca\x11\xa4\xe4|\x07H\xe1\x0c\...",b'j\xf7/\xecgQ s&[\x8a\x8e\x9e\xf5a\xe3\xc7\x1...,b'd\x98\xc6\xa7\xa3\xd22\xfd\x1d\xd3X\xa4\xa6V...,b'\x0f\xd1\x1b\x1a\x9b\x10^\x8bQ\xcdbq\x81\x8c...,b'\xc8Ai\xff\x17;A\x9fi\x9d\xf0\xf0h\x9bC\x84\...,"b""25\x81\xdbX\xc0l%\xddhh\xad\xf4\xd3H,Q\xb7o8...",b'\xfe1\xb5\x9a\xd3X\xb6\x97\x01p\xd6\xf4:\xe7...,"b""\xbd\xabz\xd7\x1c'PQ\x04\x12F\xa7\xed8F$M=j\...",b'O\xcf\xfa\x84y\xdd\xb8+j\xba\xb2R\x191\x89=V...,b'\x87<St\x0c\x87D

In [22]:
dfn.id_user=dfn.id_user.astype(str)
dfn_copy=dfn.copy()
def hashme3(id_user,month,year):
    if(id_user=="DEL"):
        return id_user
    r=ciphers[(id_user,month)]
    if( (month==12) and (year==2010)):
        F_dict[id_user][0]=r
    else:
        F_dict[id_user][month]=r
    return r

In [97]:
dfn[dfn["id_user"]=="DEL"]

,id_user,date,hours,id_item,price,qty,month,year
19,DEL,2010-12-01,,,,,12,2010
29,DEL,2010-12-01,,,,,12,2010
41,DEL,2010-12-01,,,,,12,2010
47,DEL,2010-12-01,,,,,12,2010
48,DEL,2010-12-01,,,,,12,2010
49,DEL,2010-12-01,,,,,12,2010
50,DEL,2010-12-01,,,,,12,2010
51,DEL,2010-12-01,,,,,12,2010
56,DEL,2010-12-01,,,,,12,2010
61,DEL,2010-12-01,,,,,12,2010


In [56]:
dfn.id_user=dfn.id_user.astype(str)
dfn_copy=dfn.copy()
def hashme2(id_user, month, year): 
    """
        :return: Hash[user,month] = SHA256(Salt[user,month] + id_user)
    """
    if(id_user=="DEL"):
        return id_user
    else:
        hashme2.counter+=1
        percent=(hashme2.counter/307054)*100 
        sys.stdout.write("\rProgress %i -- Count : %i / 307054" % (percent,hashme2.counter))
        sys.stdout.flush()
        if(year==2010 and month==12 ):
            if(F_dict[id_user][0] != "DEL"):
                return F_dict[id_user][0]
            r = hs.sha256(id_user.encode()).hexdigest()
            F_dict[id_user][0]=r
        else:
            if(F_dict[id_user][month] != "DEL"):
                return F_dict[id_user][month]
            r = hs.sha256(id_user.encode()).hexdigest()
            F_dict[id_user][month]=r
        return r
#Static Counter for ProgressBar
hashme2.counter=0

In [23]:
users = df["id_user"].unique()
users = users[users!="DEL"]
ciphers={}
for u in users:
    for m in range(0,14):
        ciphers[(u,m)]=users[rd.randrange(len(users)-1)]

In [78]:
ciphers

{'17850': '14205',
 '13047': '14027',
 '12583': '13854',
 '13748': '17651',
 '15100': '17172',
 '15291': '17187',
 '14688': '14741',
 '17809': '15087',
 '16098': '16709',
 '18074': '14860',
 '17420': '16978',
 '16029': '17819',
 '16250': '13483',
 '12431': '13158',
 '13705': '18218',
 '13767': '16313',
 '17924': '12642',
 '13448': '12480',
 '15862': '15101',
 '15513': '15132',
 '16218': '17429',
 '14045': '14782',
 '14307': '17306',
 '17908': '13496',
 '12838': '12697',
 '13255': '16072',
 '16583': '14030',
 '18085': '15907',
 '13758': '12956',
 '15983': '15075',
 '14849': '16048',
 '17968': '15681',
 '16210': '13507',
 '17897': '12853',
 '17377': '12622',
 '16552': '14708',
 '17181': '12693',
 '17951': '16855',
 '14729': '16079',
 '15012': '17705',
 '12868': '12517',
 '17572': '13441',
 '14078': '12456',
 '14001': '14886',
 '12662': '15865',
 '15525': '17109',
 '14237': '17384',
 '17905': '14109',
 '15485': '17383',
 '12433': '14805',
 '16955': '12987',
 '15350': '15724',
 '15605': '1

In [502]:
def hashme(id_user, month, year): 
    """
        :return: Hash[user,month] = SHA256(Salt[user,month] + id_user)
    """
    if(id_user=="DEL"):
        return id_user
    else:
        hashme.counter+=1
        percent=(hashme.counter/307054)*100 
        sys.stdout.write("\rProgress %i -- Count : %i / 307054" % (percent,hashme.counter))
        sys.stdout.flush()
        if(year==2010 and month==12 ):
            if(F_dict[id_user][0] != "DEL"):
                return F_dict[id_user][0]
            r = hs.sha256(salt_dict[id_user][0] + id_user.encode()).hexdigest()
            F_dict[id_user][0]=r
        else:
            if(F_dict[id_user][month] != "DEL"):
                return F_dict[id_user][month]
            r = hs.sha256(salt_dict[id_user][month] + id_user.encode()).hexdigest()
            F_dict[id_user][month]=r
        return r
#Static Counter for ProgressBar
hashme.counter=0

In [58]:
def generateF(df):
    users=df["id_user"].unique().tolist()
    months=list(range(1,14))
    F = pd.DataFrame(index=users,columns=months)
    for col in F.columns:
        F[col].values[:] = "DEL"
    F_dict = F.T.to_dict('list')
    
    return F,F_dict

F,F_dict = generateF(df)

In [59]:
months=list(range(1,14))
month_dict={ i : months[i] for i in range(0, len(months) ) }
F = pd.DataFrame.from_dict(F_dict,orient="index")
F = F.rename(columns=month_dict)  
F.head(3)

,1,2,3,4,5,6,7,8,9,10,11,12,13
17850,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
13047,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
12583,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL


In [26]:
%%time
def populateF():
    """
        Apply hashme for all rows
    """
    #dfn_copy.sort_values(["id_user","month"])
    #salt_table.sort_index(inplace=True)
    dfn["id_user"]=dfn.apply(lambda x: hashme3(x["id_user"],x["month"],x["year"]), axis=1)

populateF()
hashed_copy=dfn.copy()

CPU times: user 11.2 s, sys: 23.7 ms, total: 11.2 s
Wall time: 11.3 s


In [27]:
F

,1,2,3,4,5,6,7,8,9,10,11,12,13
17850,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
13047,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
12583,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
13748,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
15100,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
15291,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
14688,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
17809,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
16098,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
18074,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL


In [59]:
%%time
def populateF():
    """
        Apply hashme for all rows
    """
    dfn_copy.sort_values(["id_user","month"])
    #salt_table.sort_index(inplace=True)
    dfn["id_user"]=dfn_copy.apply(lambda x: hashme2(x["id_user"],x["month"],x["year"]), axis=1)

populateF()
hashed_copy=dfn.copy()

Progress 44 -- Count : 136226 / 307054

KeyboardInterrupt: 

In [28]:
months=list(range(1,14))
month_dict={ i : months[i] for i in range(0, len(months) ) }
F = pd.DataFrame.from_dict(F_dict,orient="index")
F = F.rename(columns=month_dict)  
F.to_csv("F_truth_final.csv",index=True)
F.head(3)

,1,2,3,4,5,6,7,8,9,10,11,12,13
17850,13911,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
13047,15646,DEL,14089,18114,DEL,15740,13587,DEL,14708,DEL,13590,16871,DEL
12583,12509,13102,17589,17365,DEL,13187,15240,12444,17545,17581,16190,17913,12509


In [183]:
%%time
def retransformDel(dfn):
    dfn_dict = dfn.T.to_dict('list')
    for i in range(0,len(dfn_dict)):
        if(dfn_dict[i][0]=="DEL"):
            dfn_dict[i][1]=""
            dfn_dict[i][2]=""
            dfn_dict[i][3]=""
            dfn_dict[i][4]=""
            dfn_dict[i][5]=""
    dfn = pd.DataFrame.from_dict(dfn_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year"}) 
    return dfn

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11 µs


In [404]:
dfn=hashed_copy.copy()
#dfn[dfn["id_user"]=="DEL"]
#dfn.to_csv("./k4nU.csv",index=False)

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
%%time
def hour_to_int(hours):
    if(isinstance(hours,float)):
        return int(hours)
def parse_dates(date):
    if(isinstance(date,dt.date) and not pd.isnull(date)):
        return date.strftime('%Y/%m/%d')
dfn["date"]=dfn["date"].astype("datetime64[ns]")
dfn["qty"]=dfn.apply(lambda x: hour_to_int(x["qty"]), axis=1)
dfn["price"]=df["price"].round(2)
dfn["date"]=dfn["date"].apply(lambda x: x.strftime('%Y/%m/%d'))
dfn["hours"]=dfn.apply(lambda x: hour_to_int(x["hours"]), axis=1)
dfn.drop(["month","year"],axis=1,inplace=True)
dfn.sort_values(["date"],inplace=True)
dfn=retransformDel(dfn)
dfn.to_csv("./k4nU_reset.csv",index=False)
dfa=pd.read_csv("./k4nU_reset.csv")
dfa

In [31]:
%%time
ground_truth, submission = round1_preprocessing(
    "./ground_truth.csv", "./k4nU_reset.csv"
)
check_format_trans_file(ground_truth, submission)

CPU times: user 13.6 s, sys: 164 ms, total: 13.8 s
Wall time: 13.8 s


In [32]:
%%time
metric = Metrics(ground_truth, submission)

CPU times: user 21min 35s, sys: 11.3 s, total: 21min 46s
Wall time: 22min 22s


In [33]:
%%time
scores = metric.scores_util()
print(scores)

[0.8742283481935404, 0, 0.5634377980873474, 0.0122347924, 0.5293282141, 0.0303]
CPU times: user 58.2 s, sys: 1.19 s, total: 59.4 s
Wall time: 53.2 s


In [50]:
pres = pd.read_csv("./k4nU2.csv", parse_dates=["date"])

In [52]:
pres[pres["id_user"]!="DEL"]

,id_user,date,hours,id_item,price,qty
14,25724c7b3c4f6e1cc81154cdb14d466fa1d9d7e8c0277c...,2010-12-01,13.0,22579,0.85,7.0
25,25724c7b3c4f6e1cc81154cdb14d466fa1d9d7e8c0277c...,2010-12-01,13.0,84970L,0.95,12.0
31,74bacd45deb708186843e597cbca3d10207bf7abd7ad46...,2010-12-01,11.0,21071,1.25,6.0
32,f62d8e71b1a07ff26b4f615c9b499bbd449836ee4a07f7...,2010-12-01,12.0,22632,2.10,48.0
36,e24602df57d00c1eb2344271bbb0d9053d94e99723ae5d...,2010-12-01,12.0,22441,2.10,1.0
...,...,...,...,...,...,...
307049,c047b9479555e98fc6c965302e028a1a509a25938cb3bf...,2011-12-09,12.0,20727,1.65,10.0
307050,6edda922efaea5a775f27100ce8bffb9b5081d4b916863...,2011-12-09,13.0,23033,1.45,6.0
307051,160bf7f74a09224629eccfdf5ae4bec49064620710d231...,2011-12-09,12.0,23298,4.95,3.0
307052,7b9a4955f082979765b55d7c131268c2ed22fe6d242cdd...,2011-12-09,12.0,22728,3.75,4.0


In [167]:
with open ('scores_k4nU', 'rb') as fp:
        object = pickle.load(fp)
print(object)

[0.8669807315791781, 0, 0.8183278676625013, 0.2189653727, 0.0, 0.2291]


In [169]:
with open ('scores_k4nU'', 'rb') as fp:
        object = pickle.load(fp)
print(object)

SyntaxError: EOL while scanning string literal (<ipython-input-169-231f6e3c4bec>, line 1)